# 步骤一：（仅carol）前置部署

在体验TrustedFlow之前，carol需要先部署CapsuleManager。CapsuleManager是TrustedFlow的核心模块之一，负责授权管理和密钥管理，CapsuleManager被设计为运行在TEE上，机构可以对CapsuleManager的代码和环境进行远程认证。

## 选项一：仿真模式部署CapsuleManager

如果你希望使用仿真模式进行体验，则可以按下列说明进行。

### 第一步：运行CapsuleManager镜像

```bash
docker run -it secretflow/capsule-manager-sim-ubuntu20.04:latest bash
```

### 第二步：启动CapsuleManager

```bash
./capsule_manager
```

1. 默认的监听端口为8888，您可以在启动时添加`--port xx`参数修改为其他端口号。
2. CapsuleManager默认启用mTLS，关于如何配置mTLS可以参考[CapsuleManager mTLS](https://github.com/secretflow/capsule-manager/blob/master/README.md#mutual-tls)。
3. 您可以添加启动参数`--enable-tls false`来关闭mTLS功能，注意关闭mTLS是不安全的，生产环境建议启动mTLS。




## 选项二：在SGX机器上运行CapsuleManager

如果您希望在SGX机器上运行，则可以按下列说明进行。

### 1. 运行镜像

```bash
docker run -it --net host -v /dev/sgx_enclave:/dev/sgx/enclave -v /dev/sgx_provision:/dev/sgx/provision --privileged=true secretflow/capsule-manager-ubuntu20.04:latest
```

### 2. 修改 PCCS 配置

> 提示：如果您还没有PCCS服务，则可以参考[部署PCCS](../architecture/tee/sgx.md#如何部署pccs服务)。


1. 修改PCCS的配置文件/etc/sgx_default_qcnl.conf，把PCCS_URL配置为PCCS的实际部署服务地址。

```bash
# PCCS server address
"pccs_url": "https://localhost:8081/sgx/certification/v4/"

# To accept insecure HTTPS certificate, set this option to FALSE
"use_secure_cert": false

```

2. 修改occlum_release/image/etc/kubetee/unified_attestation.json，将ua_dcap_pccs_url配置为实际的PCCS服务地址。

```json
{
    "ua_ias_url": "",
    "ua_ias_spid": "",
    "ua_ias_apk_key": "",
    "ua_dcap_lib_path": "",
    "ua_dcap_pccs_url": "https://localhost:8081/sgx/certification/v3/",
    "ua_uas_url": "",
    "ua_uas_app_key": "",
    "ua_uas_app_secret": ""
}
```


3. 配置TLS

我们建议您启用TLS，关于如何配置mTLS可以参考[CapsuleManager mTLS](
https://github.com/secretflow/capsule-manager/blob/master/README.md#mutual-tls)。

如果您希望关闭TLS，则可以修改occlum_release/config.yaml，配置成`enable_tls: false`则表示关闭TLS。

注意，使用TLS可以提升安全，建议您在生产环境中启用TLS功能。

### 4. 生成私钥后，使用私钥进行build。

您首先需要生成私钥，然后使用以下命令构建occlum。生成私钥可以参考下列脚本，生成的私钥保存在当前目录的private_key.pem。请妥善保存您的私钥，不要泄露给其他人。

```bash
openssl genrsa -3 -out private_key.pem 3072
```

生成公私钥后，使用私钥构建occlum。

```bash
occlum build -f --sign-key private_key.pem
```

### 5. 运行服务

执行下列命令启动服务。

> 默认的监听端口为8888，您可以在启动时添加`--port xx`参数修改为其他端口号，或者修改config.yaml文件中的`port`。

```bash
occlum run /bin/auth-manager --config_path /host/config.yaml
```

### 6. 获取CapsuleManager的mrenclave

执行下列命令可以获得CapsuleManager的mrenclave，mrenclave是表征CapsuleManager代码、数据、运行环境等的度量值，详细解释见[Enclave](../architecture/tee/sgx.md#enclave)。

输出内容为一串十六进制字符串，您可以保存下来，后续步骤会使用到。

```bash
occlum print mrenclave | tr a-z A-Z
```